In [15]:
import pandas as pd
import movingpandas as mpd
import glob
import os, json, sys
import re
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from datetime import datetime
import folium
import pickle
from folium import plugins
from folium.features import DivIcon
import geopandas
from datetime import datetime, timedelta
from tqdm import tqdm
from shapely.geometry import Polygon
import datetime as dt

In [16]:
prefix = '../../data/processed'
json_pattern = os.path.join(prefix,'aishub_linear_10S*.csv')
file_list = glob.glob(json_pattern)
print(file_list)

dfs = [] # an empty list to store the data frames
for file in tqdm(file_list):
    dfs.append(pd.read_csv(file, parse_dates=['timestamp'])) # append the data frame to the list
temp = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

['../../data/processed/aishub_linear_10S_2021_07.csv', '../../data/processed/aishub_linear_10S_2020_07.csv', '../../data/processed/aishub_linear_10S_2021_04.csv', '../../data/processed/aishub_linear_10S_2020_04.csv', '../../data/processed/aishub_linear_10S_2021_10.csv', '../../data/processed/aishub_linear_10S_2020_01.csv', '../../data/processed/aishub_linear_10S_2020_10.csv', '../../data/processed/aishub_linear_10S_2021_01.csv']


100%|██████████| 8/8 [00:11<00:00,  1.38s/it]


In [17]:
temp["type"].value_counts()

1.0     7108569
3.0     1203534
18.0     466063
2.0         155
Name: type, dtype: int64

In [18]:
temp["mmsi"] = temp["mmsi"].astype(int)

In [19]:
mmsis = temp["mmsi"].value_counts()

In [20]:
intMMSIS = mmsis.index.astype(int)

In [22]:
import requests
import time
from bs4 import BeautifulSoup#fetch link
mmssiStaticInfo = {}
default = {"shipType" : None, "length" : None, "width": None}
noInfo = 0
noLength  = 0
exs = 0
for id in tqdm(intMMSIS): 
    page = requests.get(f'https://www.myshiptracking.com/vessels?name={id}&destination=&sf=0&st=60&sort_dir=DESC&sort=RECEIVED&fbuild=1880&tbuild=2022')
    #Get HTML content
    soup = BeautifulSoup(page.text, 'html.parser')
    myspans = soup.find_all("div", {"class": "icon-cont"})
    if len(myspans) == 0:
        mmssiStaticInfo[id] = deepcopy(default)
        noInfo += 1
    else:
        vesselType = myspans[0].text
        mmssiStaticInfo[id] = deepcopy(default)
        mmssiStaticInfo[id]["shipType"] = vesselType
        try:
            ahref = soup.find(class_="pl-2", href=True)
            page = requests.get(f'https://www.myshiptracking.com{ahref["href"]}')
            #Get HTML content
            soup = BeautifulSoup(page.text, 'html.parser')
            table = soup.find('table', attrs={'class':'table-sm'})
            table_body = table.find('tbody')

            rows = table_body.find_all('tr')
            found = False
            for row in rows:
                cols = row.find('th')
                if cols.text == "Size":
                    found = True
                    size_raw = row.find('td')
                    matches = re.findall('[0-9]+', size_raw.text)
                    mmssiStaticInfo[id]["length"] = matches[0]
                    mmssiStaticInfo[id]["width"] = matches[1]
                    break
            if not found:
                noLength += 1
        
        except Exception as e:
            exs += 1
    time.sleep(0.2)
print(f'Total: {len(intMMSIS)}, noInfo: {noInfo}, noLength: {noLength}, exs: {exs}')

with open('mmssiStaticInfo.pkl', 'wb') as f:
    pickle.dump(mmssiStaticInfo, f)
    
mmssiStaticInfo


  4%|▍         | 97/2541 [01:58<3:40:57,  5.42s/it]

In [234]:
from bisect import bisect

tides = pd.read_csv("../../data/tide_data/combined_tides.csv", parse_dates=["timestamp"])
tides.sort_values(by=["timestamp"], inplace=True)
temp["tide_level"] = -1.0
temp["tide"] = -1.0
temp["tide_timestamp"] = None
i = 0
for year in [2020]:
   for month in [1, 4, 7, 10]:
      for day in tqdm(range(1,32)):
         for hour in range(0,24):
            subset_indexes = temp[(temp["timestamp"].dt.year == year) & (temp["timestamp"].dt.month == month) & (temp["timestamp"].dt.day == day) & (temp["timestamp"].dt.hour == hour)].index
            tide_subset = tides[(tides['timestamp'].dt.year ==year) & (tides["timestamp"].dt.month == month) & (tides["timestamp"].dt.day == day) & (tides["timestamp"].dt.hour == hour)]
            for idx in subset_indexes:
               if len(tide_subset["timestamp"]) > 0:
                  i_index = min(tide_subset["timestamp"], key=lambda d: abs(d - temp.at[idx,"timestamp"]))
               else:
                  i_index = min(tides[(tides['timestamp'].dt.year ==year) & (tides["timestamp"].dt.month == month) & (tides["timestamp"].dt.day == day)]["timestamp"], key=lambda d: abs(d - temp.at[idx,"timestamp"]))
                 
               tide_row = tides[tides['timestamp'] == i_index]
               temp.at[idx, "tide_level"] = float(tide_row["level"].values[0])
               temp.at[idx, "tide"] = float(tide_row["tide"].values[0])
               temp.at[idx, "tide_timestamp"] = tide_row["timestamp"].values[0]
        
      

100%|██████████| 31/31 [27:04<00:00, 52.40s/it] 


In [235]:
temp.to_csv("../../data/usable/tide_info.csv", index=False)

In [7]:
temp =  pd.read_csv('../../data/usable/tide_info.csv')

In [8]:
with open('mmssiStaticInfo.pkl', 'rb') as f:
    mmssiStaticInfo = pickle.load(f)

In [9]:
k = dict([(mmsi, item["shipType"]) for mmsi, item in mmssiStaticInfo.items()])
temp["shipType"]= temp['mmsi'].map(k)

In [10]:
temp["shipType"].unique()

array([nan, 'Pilot Ship', 'Pilot Vessel', 'Passenger'], dtype=object)

In [ ]:
bigShips_no_pass = temp[temp["shipType"].isin(['Bulk Carrier', 'Bunkering Tanker',
       'Cargo', 'Cargo A', 'Cargo B', 'Cargo C',
       'Cargo D', 'Cement Carrier', 'Chemical Tanker', 'Container Ship',
       'Edible Oil Tanker', 'General Cargo',
       'Heavy Lift Vessel', 'Heavy Load Carrier','Lpg Tanker', 
       'Oil Products Tanker', 'Oil/Chemical Tanker','Ro-Ro Cargo',
        'Suction Dredger', 'Tanker',
       'Tanker A', 'Tanker B', 'Trailing Suction Hopper Dredger','Vehicles Carrier', 'Waste Disposal Vessel'])]
print(bigShips_no_pass.groupby('mmsi').count().shape[0])
print(bigShips_no_pass.groupby('traj_id').count().shape[0])

In [ ]:
bigShips_no_pass.to_csv("../../data/usable/aishub_linear_artificial_big_ships_no_passenger.csv", index=False)

In [ ]:
from statistics import mean

temp[temp["tide_level"] > 0]

In [52]:
shipTypesArray = [item["shipType"] for mmsi, item in mmssiStaticInfo.items()]

In [55]:
unique, counts = np.unique(shipTypesArray, return_counts=True)
unique

array(['Passenger', 'Pilot Ship', 'Pilot Vessel'], dtype='<U12')

In [ ]:
import seaborn as sns
import plotting_config
x_axis = np.arange(0, 20, 0.001)
sns.set_style("whitegrid")
k = shipTypesArray
SMALL_SIZE = 22
MEDIUM_SIZE = 22
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
unique, counts = np.unique(k, return_counts=True)
inds = counts.argsort()[-15:]
sortedUnique = unique[inds]
sortedCounts = counts[inds]
plt.figure(figsize=(10,6))
plt.barh(sortedUnique, sortedCounts)
plt.ylabel("Ship Type")
plt.xlabel("Unique Ships Encountered")
plt.savefig("shipTypes.svg", format="svg")
print(sortedUnique)

In [26]:
temp.to_csv("../../data/usable/aishub_linear_artificial_whole.csv", index=False)

In [8]:
bigShips = temp[temp["shipType"].isin(['Bulk Carrier', 'Bunkering Tanker',
       'Cargo', 'Cargo A', 'Cargo B', 'Cargo C',
       'Cargo D', 'Cement Carrier', 'Chemical Tanker', 'Container Ship',
       'Edible Oil Tanker', 'General Cargo',
       'Heavy Lift Vessel', 'Heavy Load Carrier','Lpg Tanker', 
       'Oil Products Tanker', 'Oil/Chemical Tanker', 'Passenger', 'Passengers Ship', 'Ro-Ro Cargo',
       'Ro-Ro/Passenger Ship', 'Suction Dredger', 'Tanker',
       'Tanker A', 'Tanker B', 'Trailing Suction Hopper Dredger','Vehicles Carrier', 'Waste Disposal Vessel'])]

In [10]:
a = temp[temp["shipType"] == 'Fishing']
a['mmsi'].value_counts()

211749530.0    12056
211618000.0     6633
211164000.0     5482
211639000.0     4785
218799000.0     2464
246287000.0      898
211608000.0      220
211392000.0      178
211438440.0      124
211793660.0       60
Name: mmsi, dtype: int64

In [9]:
print(bigShips.groupby('mmsi').count().shape[0])
print(bigShips.groupby('traj_id').count().shape[0])

1074
11381


In [34]:
bigShips.to_csv("../../data/usable/aishub_linear_artificial_big_ships.csv", index=False)

In [12]:
bigShips_no_pass = temp[temp["shipType"].isin(['Bulk Carrier', 'Bunkering Tanker',
       'Cargo', 'Cargo A', 'Cargo B', 'Cargo C',
       'Cargo D', 'Cement Carrier', 'Chemical Tanker', 'Container Ship',
       'Edible Oil Tanker', 'General Cargo',
       'Heavy Lift Vessel', 'Heavy Load Carrier','Lpg Tanker', 
       'Oil Products Tanker', 'Oil/Chemical Tanker','Ro-Ro Cargo',
        'Suction Dredger', 'Tanker',
       'Tanker A', 'Tanker B', 'Trailing Suction Hopper Dredger','Vehicles Carrier', 'Waste Disposal Vessel'])]

In [13]:
print(bigShips_no_pass.groupby('mmsi').count().shape[0])
print(bigShips_no_pass.groupby('traj_id').count().shape[0])

1045
5618


In [ ]:
bigShips_no_pass.to_csv("../../data/usable/aishub_linear_artificial_big_ships_no_passenger.csv", index=False)

In [35]:
cargo = temp[temp["shipType"].isin(['Bulk Carrier', 'Cargo', 'Cargo A', 'Cargo B', 'Cargo C',
       'Cargo D', 'Cement Carrier','Container Ship','General Cargo', 'Heavy Load Carrier',
       'Ro-Ro Cargo', 'Vehicles Carrier', 'Waste Disposal Vessel'])]

In [36]:
print(cargo.groupby('mmsi').count().shape[0])
print(cargo.groupby('traj_id').count().shape[0])

970
4065


In [37]:
cargo.to_csv("../../data/usable/aishub_linear_artificial_cargo.csv", index=False)

In [55]:
print(cargo["lat"].min())
print(cargo["lat"].max())
print(cargo["lon"].min())
print(cargo["lon"].max())

53.49
53.716475
8.372
8.586853333333334


In [8]:
tanker = temp[temp["shipType"].isin(['Bunkering Tanker','Chemical Tanker', 
       'Edible Oil Tanker','Lpg Tanker', 
       'Oil Products Tanker', 'Oil/Chemical Tanker', 'Tanker',
       'Tanker A', 'Tanker B'])]

In [9]:
print(tanker.groupby('mmsi').count().shape[0])
print(tanker.groupby('traj_id').count().shape[0])

71
1194


In [40]:
tanker.to_csv("../../data/usable/aishub_linear_artificial_tanker.csv", index=False)

In [14]:
path_to_json = '../../data/ais-hub/202*'

json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

dfs = [] # an empty list to store the data frames
for file in tqdm(file_list):
    data = pd.read_json(file, lines=True) # read data frame from json file
    data['absolute-time'] = pd.json_normalize(data['meta-data'])['absolute-time']
    dfs.append(data) # append the data frame to the list

temp = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

 10%|▉         | 1821/18854 [00:35<05:33, 51.12it/s]


KeyboardInterrupt: 

## Preprocessing dataframe

In [3]:
df = temp[['longitude', 'latitude', 'absolute-time', 'source-mmsi', 'speed-over-ground',  'course-over-ground', \
          'true-heading', 'type' ]]
#'navigational-status'
#'position-accuracy',
#'rate-of-turn',
# remove rows with NaNs (just 'navigational-status' and 'rate-of-turn')
df = df.dropna()
df = df.rename(columns={'longitude': 'lon', 'latitude': 'lat', 'absolute-time': 'timestamp', 'source-mmsi': 'mmsi',\
    'speed-over-ground': 'sog', 'course-over-ground': 'cog',\
    'true-heading': 'heading'})


In [4]:
len(df["type"])

562

In [5]:
df["type"].value_counts()

1     378
3     154
18     30
Name: type, dtype: int64

In [46]:
#df["nav_status"].value_counts()

In [47]:
df["mmsi"].value_counts()

220486000    32854
211595610    30316
211599340    29940
219285000    29391
245586000    29140
             ...  
970016504        1
970220116        1
970216811        1
605854626        1
970016780        1
Name: mmsi, Length: 802, dtype: int64

In [48]:
print(f'Lon min: {df["lon"].min()}')
print(f'Lon max: {df["lon"].max()}')
print(f'Lat min: {df["lat"].min()}')
print(f'Lat min: {df["lat"].max()}')

Lon min: 8.372
Lon max: 8.592153333333334
Lat min: 53.49
Lat min: 53.71996333333333


In [4]:
# remove "standing still in habour" and ship moving too slow
df = df[df.sog>3]
df = df[(df.lon<180) | (df.lat<90)]

## Filter unrealistic sog values (over 100 for example)
df = df[df.sog<=30]

# convert to GeoDataFrame
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(x=df.lon, y=df.lat), crs="WGS84"
)
gdf['time'] = pd.to_datetime(gdf['timestamp'], unit='ms')
gdf = gdf.set_index('time')

In [5]:
gdf

,lon,lat,timestamp,mmsi,sog,cog,heading,type,geometry
time,,,,,,,,,
2020-01-06 06:50:27.049,8.574858,53.535763,1578293427049,211225940,7.100000,262.1,268,3,POINT (8.57486 53.53576)
2020-01-06 06:50:36.362,8.540535,53.567770,1578293436362,218709000,11.400001,334.0,336,1,POINT (8.54054 53.56777)
2020-01-06 06:50:54.603,8.536817,53.573850,1578293454603,211411980,11.100000,334.2,336,1,POINT (8.53682 53.57385)
2020-01-06 06:51:04.725,8.554048,53.552922,1578293464725,211221570,7.100000,330.5,332,3,POINT (8.55405 53.55292)
2020-01-06 06:51:39.758,8.572027,53.537347,1578293499758,211225940,7.600000,328.0,326,3,POINT (8.57203 53.53735)
...,...,...,...,...,...,...,...,...,...
2020-01-06 07:18:29.141,8.439600,53.638517,1578295109141,211411980,6.300000,319.9,280,3,POINT (8.43960 53.63852)
2020-01-06 07:18:45.802,8.505863,53.603110,1578295125802,211221570,8.300000,319.4,319,1,POINT (8.50586 53.60311)
2020-01-06 07:18:54.478,8.511022,53.599765,1578295134478,211225940,10.200000,326.4,320,3,POINT (8.51102 53.59976)


## Generate Trajectory Collection with MovingPandas

In [5]:
# Specify minimum length for a trajectory (in meters)
minimum_length = 1500 
collection = mpd.TrajectoryCollection(gdf, 'mmsi', min_length=minimum_length)

In [6]:
# (optional) add calculated speed and direction direct from the lat/lon position of the vessel.
# This should not be necessary as we will use the pure interpolated real values!

collection.add_speed(overwrite=True)
collection.add_direction(overwrite=True)

In [9]:
collection.trajectories[1].df

,lon,lat,timestamp,mmsi,sog,cog,heading,type,geometry,speed,direction
time,,,,,,,,,,,
2020-01-06 06:50:27.049,8.574858,53.535763,1578293427049,211225940,7.100000,262.10000,268,3,POINT (8.57486 53.53576),3.541386,-46.812669
2020-01-06 06:51:39.758,8.572027,53.537347,1578293499758,211225940,7.600000,328.00000,326,3,POINT (8.57203 53.53735),3.541386,-46.812669
2020-01-06 06:53:06.262,8.569125,53.540050,1578293586262,211225940,8.200000,327.50000,325,1,POINT (8.56912 53.54005),4.128324,-32.593602
2020-01-06 06:54:17.028,8.566517,53.542502,1578293657028,211225940,9.400001,329.50000,326,1,POINT (8.56652 53.54250),4.564892,-32.362267
2020-01-06 06:55:26.770,8.563860,53.545170,1578293726770,211225940,9.600000,330.30002,328,1,POINT (8.56386 53.54517),4.950652,-30.667608
2020-01-06 06:56:46.360,8.560922,53.548405,1578293806360,211225940,10.100000,332.30002,328,1,POINT (8.56092 53.54841),5.143233,-28.410523
2020-01-06 07:01:06.684,8.551428,53.558737,1578294066684,211225940,9.500000,330.80002,329,1,POINT (8.55143 53.55874),5.035099,-28.682122
2020-01-06 07:03:51.956,8.544292,53.564915,1578294231956,211225940,9.400001,324.20000,325,3,POINT (8.54429 53.56491),5.049510,-34.514046
2020-01-06 07:07:12.226,8.537037,53.572477,1578294432226,211225940,9.100000,329.00000,327,3,POINT (8.53704 53.57248),4.839354,-29.729005


In [7]:
# Remove Trajectories that have too long gaps in consecutive AIS signals
trips = mpd.ObservationGapSplitter(collection).split(gap=timedelta(minutes=5))
# Remove all anchoring chips with a tolerance of 15 diameter (for example the Tugs laying in the "Schlepperhafen")
trips = mpd.StopSplitter(trips).split(max_diameter=15, min_duration=timedelta(minutes=3), min_length=1500)
# Outlier detecting and cleaning (Outlier (interquantile range - iqr) based cleaner.)
# From moving pandas: "Note: Setting alpha=3 is widely used."
trips = mpd.OutlierCleaner(trips).clean({'speed': 3})

In [54]:
# hand-made polygon of the area of interest which excludes the inner harbour and floodgates
# (basically just the Weser)
area_of_intereset = Polygon([(8.4867561,53.486203), (8.4867923,53.4861983), (8.5063362,53.4849813), (8.5257339,53.4990238), (8.5642719,53.5098972), \
(8.5717392,53.51939), (8.574357,53.5290337), (8.5766745,53.5363798), (8.576932,53.5390577), (8.5747862,53.5414678), \
(8.5707736,53.5437884), (8.5686493,53.5459176), (8.561182,53.5540766), (8.5582638,53.5545865), (8.5471916,53.566669), \
(8.5452175,53.5694215), (8.5317421,53.5859837), (8.5214424,53.6045765), (8.5198975,53.6112475), (8.501358,53.6405673), \
(8.4965515,53.6747479), (8.6160278,53.8760117), (8.0900574,53.9019109), (8.1202698,53.6373105), (8.3262634,53.6092107), \
(8.3468628,53.5937274), (8.4555244,53.5527509), (8.5322571,53.5392873), (8.5497665,53.5361247), (8.5537148,53.5312274), \
(8.5533714,53.5264316), (8.5487366,53.5233702), (8.5336304,53.5176549), (8.5180092,53.5108159), (8.5027313,53.5016276), \
(8.4867561,53.486203)])

# TODO not working at all (returns 0 trajectories)
# clipped_trips = trips.clip(area_of_intereset)

In [55]:
#trips.trajectories[2].plot(column="speed", linewidth=5, capstyle='round', legend=True)

In [8]:
def resample_and_interpolate(trips, resample_interval="5S", interpolate_method="linear"):
    out = pd.DataFrame()

    for t in tqdm(trips.trajectories):
        tmp = t.df.resample(resample_interval, origin="end").mean()
        tmp["mmsi"] = tmp["mmsi"].interpolate(method="bfill")
        tmp["type"] = tmp["type"].interpolate(method="bfill")
        tmp["sog"] = tmp["sog"].interpolate(method=interpolate_method)
        tmp["cog"] = tmp["cog"].interpolate(method=interpolate_method)
        tmp["lat"] = tmp["lat"].interpolate(method=interpolate_method)
        tmp["lon"] = tmp["lon"].interpolate(method=interpolate_method)
        tmp["heading"] = tmp["heading"].interpolate(method=interpolate_method)
        tmp["speed"] = tmp["speed"].interpolate(method=interpolate_method)
        tmp["direction"] = tmp["direction"].interpolate(method=interpolate_method)
        tmp["traj_id"] = str(tmp["mmsi"][0]) + str(tmp.iloc[0].name)
        tmp["timestamp"] = tmp.index
        tmp = tmp.reset_index(drop=True)
        out = out.append(tmp, ignore_index=True)
        
    return out

In [57]:
# 01: 886858
# 04: 1770463
# 07:

In [58]:
trips

TrajectoryCollection with 9198 trajectories

In [9]:
linear_out = resample_and_interpolate(trips, resample_interval='10S', interpolate_method='linear')
linear_out

100%|██████████| 7825/7825 [14:51<00:00,  8.78it/s]


,lon,lat,timestamp,mmsi,sog,cog,heading,type,speed,direction,traj_id
0,8.543378,53.565188,2020-10-09 16:27:56.972,205203190.0,5.100000,324.200000,511.0,3.0,2.798256,-31.117742,205203190.02020-10-09 16:27:56.972000
1,8.543356,53.565210,2020-10-09 16:27:57.972,205203190.0,5.111667,324.301667,511.0,3.0,2.798256,-31.117742,205203190.02020-10-09 16:27:56.972000
2,8.543334,53.565232,2020-10-09 16:27:58.972,205203190.0,5.123333,324.403334,511.0,3.0,2.798256,-31.117742,205203190.02020-10-09 16:27:56.972000
3,8.543312,53.565254,2020-10-09 16:27:59.972,205203190.0,5.135000,324.505001,511.0,3.0,2.798256,-31.117742,205203190.02020-10-09 16:27:56.972000
4,8.543290,53.565275,2020-10-09 16:28:00.972,205203190.0,5.146667,324.606668,511.0,3.0,2.798256,-31.117742,205203190.02020-10-09 16:27:56.972000
...,...,...,...,...,...,...,...,...,...,...,...
12384625,8.553637,53.558005,2020-10-12 07:41:49.607,982320197.0,5.669231,329.553846,511.0,18.0,4.769198,135.578173,982320197.02020-10-12 07:32:11.607000
12384626,8.553681,53.557971,2020-10-12 07:41:50.607,982320197.0,5.526923,332.165385,511.0,18.0,4.781540,137.470556,982320197.02020-10-12 07:32:11.607000
12384627,8.553724,53.557936,2020-10-12 07:41:51.607,982320197.0,5.384615,334.776923,511.0,18.0,4.793882,139.362939,982320197.02020-10-12 07:32:11.607000
12384628,8.553768,53.557901,2020-10-12 07:41:52.607,982320197.0,5.242308,337.388462,511.0,18.0,4.806224,141.255322,982320197.02020-10-12 07:32:11.607000


In [10]:
linear_out.to_csv("../../data/processed/aishub_linear_20_21.csv", index=False)